In [1]:
#import libraries
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

There were a few APIs at the time of this notebook.
Directly at pypi
- https://wiki.python.org/moin/PyPIJSON
- In the format https://pypi.python.org/pypi/<package_name>/json

There is also https://pypistats.org/
 - They ask that any large amount of API access go through Google Big Query

Google Big Query
- https://cloud.google.com/docs/authentication/getting-started
- https://bigquery.cloud.google.com/table/the-psf:pypi.downloads


In [2]:
# May need to install these
#!pip install jk_pypiorgapi
#!pip install pyarrow

## Authenticate and Construct Google Big Query Object

In [5]:
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key
#                  file.
key_path = "msdscapstone-db80de77b383.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

## Pypi Distribution Metadata Statistics and Data Download Queries

In [ ]:
# Find schema for thsi table here: https://docs.google.com/document/d/1mtZD2iw0TmzfNepCABdMCkhk-jwLgdgFP9awSkccwaw/edit#

In [3]:
#This query gives number of rows in the distribution meta data table
query = """
    SELECT Count(*) 
    FROM `the-psf.pypi.distribution_metadata`
    WHERE license is NOT NULL
"""
query_job = client.query(query)  # Make an API request.
results = query_job.result()  # Waits for job to complete.
#iterate through results object to fetch rows.
for row in results:
    row = str(row)[5:].split(',')[0]
    print(f"Row count with licenses = {row}")

Row count with licenses = 4223089


In [55]:
#This query gives number of rows in the distribution meta data table which have licenses and have the home_page in github
query = """
    SELECT count(*)
    FROM `the-psf.pypi.distribution_metadata`
    WHERE (lower(license)!='none' or lower(license)!='no license' or license is NOT NULL) and
    version NOT LIKE '%%dev%%' and home_page LIKE '%%github%%    
"""
query_job = client.query(query)  # Make an API request.
print("The query data:")
results = query_job.result()  # Waits for job to complete.
#iterate through results object to fetch rows.
for row in results:
    row = str(row)[5:].split(',')[0]
    print(f"rows with license and with home page in github = {row}")

The query data:
rows with license and with home page in github = 2861425


In [ ]:
#For understanding more on handling Array String data types in bigquery: 
# https://cloud.google.com/bigquery/docs/reference/standard-sql/array_functions#array_length
#https://cloud.google.com/spanner/docs/array_functions
#https://stackoverflow.com/questions/19413901/splitting-a-string-column-in-bigquery

In [22]:
#This query gives number of rows in the distribution meta data table which have licenses and have the home_page in github and also have dependencies
query = """
    SELECT p.name, p.version, p.author, p.author_email, p.license, p.home_page, dependency
    --, ARRAY_TO_STRING(p.requires_dist,',') as dependency
    FROM `the-psf.pypi.distribution_metadata` as p
    CROSS JOIN UNNEST(p.requires_dist) as dependency
    WHERE (lower(license) <> 'none' or lower(license) <> 'no license' or license is NOT NULL) and
    version NOT LIKE '%%dev%%' and home_page LIKE '%%github%%' and ARRAY_LENGTH(requires_dist)<>0
    --LIMIT 10
"""
query_job = client.query(query)  # Make an API request.
results = query_job.result()  # Waits for job to complete.


In [23]:
%%time
dataframe_meta_with_license_dependecies = results.to_dataframe(
    # Optionally, explicitly request to use the BigQuery Storage API. As of
    # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
    # API is used by default.
    #create_bqstorage_client=True,
)

CPU times: user 2min 7s, sys: 10.6 s, total: 2min 17s
Wall time: 9min 42s


In [24]:
dataframe_meta_with_license_dependecies.tail(10)

,name,version,author,author_email,license,home_page,dependency
8886181,gas-dynamics,0.3.2,Fernando de la Fuente,FernandoAdelaFuente@gmail.com,MIT,http://github.com/fernancode/gas_dynamics,numpy (==1.19.3)
8886182,opensurfacesim,0.1.3,Mark Shui Hu,watermarkhu@gmail.com,BSD-3,https://github.com/watermarkhu/opensurfacesim,networkx (>=2.0)
8886183,husqvarna-getl,1.8.0,Linus Wallin,linus.wallin@husqvarnagroup.com,MIT,https://github.com/husqvarnagroup/GETL/,"pyspark (>=2.4.4,<3.1)"
8886184,tqp,0.5.0,Giacomo Tagliabue,giacomo@gmail.com,MIT,https://github.com/4Catalyzer/tqp,boto3
8886185,dmm-api,0.1.1,Takeru Saito,takelushi@gmail.com,MIT,https://github.com/takelushi/dmm-api-py,"requests (>=2.25.0,<3.0.0)"
8886186,evogression,0.6.4,Zach Bateman,None,MIT,https://github.com/zachbateman/evogression.git,tqdm
8886187,pymap-admin,0.5.3,Ian Good,ian@icgood.net,MIT,https://github.com/icgood/pymap-admin/,protobuf
8886188,pymap,0.22.2,Ian Good,ian@icgood.net,MIT,https://github.com/icgood/pymap/,proxy-protocol (~=0.5.5)
8886189,code-video-generator,0.2.0,Don Brown,dbrown@sleuth.io,APLv2 License,https://github.com/sleuth-io/code-video-generator,pyglet (>=1.5.8)
8886190,code-video-generator,0.2.3,Don Brown,dbrown@sleuth.io,APLv2 License,https://github.com/sleuth-io/code-video-generator,manimce (>=0.1.0)


In [25]:
#some stats
print('All=',dataframe_meta_with_license_dependecies.shape)
print('with licenses=',dataframe_meta_with_license_dependecies[dataframe_meta_with_license_dependecies.license.isna()==False].shape)
print('with authors=',dataframe_meta_with_license_dependecies[dataframe_meta_with_license_dependecies.author.isna()==False].shape)
print('with author_email=',dataframe_meta_with_license_dependecies[dataframe_meta_with_license_dependecies.author_email.isna()==False].shape)

All= (8886191, 7)
with licenses= (8886191, 7)
with authors= (8615637, 7)
with author_email= (7665918, 7)


In [26]:
#changing all text to lower for easy querying
df_pypi_meta_with_license = dataframe_meta_with_license_dependecies.apply(lambda x: x.astype(str).str.lower())

In [27]:
#save to csv
df_pypi_meta_with_license.to_csv('pypi_meta_with_license_dependecies.csv')

# Sample query to pull Downloads data

In [ ]:
#https://medium.com/google-cloud/how-to-work-with-array-and-structs-in-bigquery-9c0a2ea584a6

In [ ]:
%%time
#This query pulls data from downloads table

query_downloads = """
    SELECT  country_code, file.project as name, file.version as version, Count(file.project) as num_downloads
    FROM `bigquery-public-data.pypi.file_downloads`
    WHERE
    DATE(timestamp)
    BETWEEN DATE_TRUNC(DATE_SUB(DATE('2020-01-01'), INTERVAL 365 DAY), DAY)
    AND DATE('2020-01-01')--CURRENT_DATE()
    GROUP BY country_code, file.project, file.version
    ORDER BY Count(file.project) DESC
    --LIMIT 5 --use for checking sample data before running whole query
"""
query_job_dl = client.query(query_downloads)  # Make an API request.
results_downloads = query_job_dl.result()  # Waits for job to complete.

In [ ]:
%%time
dataframe_downloads = results_downloads.to_dataframe(
    # Optionally, explicitly request to use the BigQuery Storage API. As of
    # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
    # API is used by default.
    #create_bqstorage_client=True,
)

In [ ]:
%%time
dataframe_downloads.shape

In [ ]:
dataframe_downloads.to_csv('pypi_downloads_DAY_1028201.csv')

## EDA

In [2]:
#if we need to read data for eda
#downloads = pd.read_csv('pypi_downloads_DAY_1028201.csv')

,Unnamed: 0,country_code,name,version,num_downloads
0,0,CA,six,1.16.0,94023511
1,1,CA,python-dateutil,2.8.2,93841598
2,2,CA,urllib3,1.26.7,93715131
3,3,CA,jmespath,0.10.0,93546532
4,4,CA,s3transfer,0.5.0,93111389
...,...,...,...,...,...
39930351,39930351,KR,scrapy-wayback,1.0.2,1
39930352,39930352,KR,loge,0.2.3,1
39930353,39930353,KR,limix-plot,0.0.7,1
39930354,39930354,IS,rkd-pythonic,2.4.0.0b2.dev99,1
